In [1]:
import pandas as pd
from datetime import datetime, timedelta
import json

In [2]:
!mkdir -p sailboat_attempts
!mkdir -p microsailboat_attempts

## Small boats

In [3]:
import json
with open('attempts.json') as f:
    attempts_times = json.load(f)['teams']

In [4]:
attempts_times['AberSailbot']

[{'start': '2018-08-30T15:09:00+0100',
  'end': '2018-08-30T15:25:00+0100',
  'interruptions': [],
  'attempt': 1},
 {'start': '2018-08-30T16:35:00+0100',
  'end': '2018-08-30T17:06:00+0100',
  'interruptions': [],
  'attempt': 2}]

In [5]:
# team, attempt no, source file
micro_attempts = [
    ("Plymouth (Micro-Sailboat)", 1, "Plymouth1.csv"),
    ("Plymouth (Micro-Sailboat)", 2, "Plymouth1.csv"),
    ("Plymouth (Micro-Sailboat)", 3, "Plymouth1.csv"),
    ("Plymouth (Micro-Sailboat)", 4, "Plymouth1.csv"),
    ("ZMART", 1, "ZMART1.csv"),  # We seem to be missing data during the attempt
    ("Southampton", 1, "Southampton1.csv"),
    ("Southampton", 2, "Southampton2.csv"),  # Missing the first few minutes, but...
    ("AberSailbot", 1, "Aberystwyth1.csv"),
    ("AberSailbot", 2, "Aberystwyth2.csv"),
]

def process_attempt(teamname, attemptno, sourcefile, to_folder):
    print(teamname, attemptno)
    track_data = pd.read_csv(sourcefile, parse_dates=[0])
    print(len(track_data), "points in file")
    attempt_times = attempts_times[teamname][attemptno - 1]
    start_t = datetime.strptime(attempt_times['start'], '%Y-%m-%dT%H:%M:%S%z')
    end_t  = datetime.strptime(attempt_times['end'], '%Y-%m-%dT%H:%M:%S%z')
    print("Attempt duration:", end_t - start_t)
    in_attempt = (track_data.datetime >= start_t) & (track_data.datetime <= end_t)
    print(in_attempt.sum(), "points in attempt")
    
    # Now subtract interruptions
    intrpts_n = 0
    in_intrpt = track_data.datetime < '2000-01-01'  # Start with all False, add interruptions
    for intrpt in attempt_times['interruptions']:
        int_start_t = datetime.strptime(intrpt['from'], '%Y-%m-%dT%H:%M:%S%z')
        int_end_t = datetime.strptime(intrpt['to'], '%Y-%m-%dT%H:%M:%S%z')
        if (int_end_t - int_start_t) <= timedelta(minutes=2):
            # Ignore interruptions of 2 minutes or less
            continue
        
        intrpts_n += 1
        in_this_intrpt = (track_data.datetime > int_start_t) & (track_data.datetime < int_end_t)
        in_intrpt |= in_this_intrpt
    
    print('Subtracting', intrpts_n, 'interruptions')
    track_data = track_data[in_attempt & ~in_intrpt]
    print(len(track_data), "points remaining")
    filename = '{}/{}{}.csv'.format(to_folder, teamname.split()[0], attemptno)
    track_data.to_csv(filename, index=False)
    print("Written", filename)
    print()

for teamname, attemptno, sourcefile in micro_attempts:
    process_attempt(teamname, attemptno, sourcefile, 'microsailboat_attempts')

Plymouth (Micro-Sailboat) 1
21360 points in file
Attempt duration: 0:11:00
634 points in attempt
Subtracting 0 interruptions
634 points remaining
Written microsailboat_attempts/Plymouth1.csv

Plymouth (Micro-Sailboat) 2
21360 points in file
Attempt duration: 0:06:00
331 points in attempt
Subtracting 0 interruptions
331 points remaining
Written microsailboat_attempts/Plymouth2.csv

Plymouth (Micro-Sailboat) 3
21360 points in file
Attempt duration: 0:08:00
460 points in attempt
Subtracting 0 interruptions
460 points remaining
Written microsailboat_attempts/Plymouth3.csv

Plymouth (Micro-Sailboat) 4
21360 points in file
Attempt duration: 0:14:00
804 points in attempt
Subtracting 0 interruptions
804 points remaining
Written microsailboat_attempts/Plymouth4.csv

ZMART 1
190 points in file
Attempt duration: 0:12:00
0 points in attempt
Subtracting 0 interruptions
0 points remaining
Written microsailboat_attempts/ZMART1.csv

Southampton 1
7010 points in file
Attempt duration: 0:17:00
976 point

In [6]:
sailboat_attempts = [
    ("ENSTA", 1, "ENSTA1.csv"),
    ("ENSTA", 2, "ENSTA1.csv"),
    ("SJTU", 1, "SJTU2.csv"),
    ("SJTU", 2, "SJTU2.csv"),
]

for teamname, attemptno, sourcefile in sailboat_attempts:
    process_attempt(teamname, attemptno, sourcefile, 'sailboat_attempts')

ENSTA 1
7750 points in file
Attempt duration: 0:05:00
285 points in attempt
Subtracting 0 interruptions
285 points remaining
Written sailboat_attempts/ENSTA1.csv

ENSTA 2
7750 points in file
Attempt duration: 0:50:00
2782 points in attempt
Subtracting 0 interruptions
2782 points remaining
Written sailboat_attempts/ENSTA2.csv

SJTU 1
9010 points in file
Attempt duration: 0:18:00
1038 points in attempt
Subtracting 0 interruptions
1038 points remaining
Written sailboat_attempts/SJTU1.csv

SJTU 2
9010 points in file
Attempt duration: 0:53:00
3037 points in attempt
Subtracting 0 interruptions
3037 points remaining
Written sailboat_attempts/SJTU2.csv

